# Imports

In [46]:
import os

import pandas as pd
from ray.tune import Analysis

from rayTune_common.constants import metric, mode
from rayTune_common.test import test_model
from rayTune_common.utils import config_to_model

Analyse ray tune logs

In [47]:
data = []
path_to_run_results = "/home/knut/Documents/project/run_results/run6"
path_to_csv = os.path.join(path_to_run_results, "results.csv")
path_to_config_csv = os.path.join(path_to_run_results, "config.csv")

list_experiments = [f.path for f in os.scandir(path_to_run_results) if f.is_dir()]
list_experiments.sort(key=lambda x: x.split("_")[-1])

for experient_number, path_to_experiment in enumerate(list_experiments):
    print(f"Experiment: {experient_number} -- {path_to_experiment}")
    experiment_data = {}

    best_trial_analysis = Analysis(path_to_experiment, default_metric=metric, default_mode=mode)
    best_trial_config = best_trial_analysis.get_best_config(metric=metric, mode=mode)
    best_trial_logdir = best_trial_analysis.get_best_logdir(metric=metric, mode=mode)
    list_best_trial_checkpoints = [f.path for f in os.scandir(best_trial_logdir) if f.is_dir()]
    list_best_trial_checkpoints.sort(key=lambda x: int(x.split("_")[-1]))
    best_trial_checkpoint_path = os.path.join(list_best_trial_checkpoints[-1], "checkpoint")
    best_trial_model = config_to_model(config=best_trial_config, checkpoint_path=best_trial_checkpoint_path)
    best_trial_mse = test_model(model=best_trial_model, batch_size=best_trial_config["batch_size"])
    print(f'mse: {best_trial_mse}, trial: {best_trial_logdir.split("_")[4]}')

    list_experiment_trials = [f.path for f in os.scandir(path_to_experiment) if f.is_dir()]
    list_experiment_trials.sort(key=lambda x: int(x.split("_")[4]))

    for trial_number, path_to_trial in enumerate(list_experiment_trials):
        list_trial_checkpoints = [f.path for f in os.scandir(path_to_trial) if f.is_dir()]
        list_trial_checkpoints.sort(key=lambda x: int(x.split("_")[-1]))

        trial_checkpoint_path = os.path.join(list_trial_checkpoints[-1], "checkpoint")

        trial_analysis = Analysis(path_to_trial, default_metric=metric, default_mode=mode)
        trial_config = trial_analysis.get_best_config(metric=metric, mode=mode)

        trial_model = config_to_model(config=trial_config, checkpoint_path=trial_checkpoint_path)

        trial_mse = test_model(model=trial_model, batch_size=trial_config["batch_size"])
        experiment_data[trial_number] = trial_mse

    sorted_experiment_data = dict(sorted(experiment_data.items()))
    data.append(sorted_experiment_data)

Experiment: 0 -- /home/knut/Documents/project/run_results/run6/xp_000
mse: 1.5655752420425415, trial: 26
Experiment: 1 -- /home/knut/Documents/project/run_results/run6/xp_001
mse: 2.4260880947113037, trial: 87
Experiment: 2 -- /home/knut/Documents/project/run_results/run6/xp_002
mse: 7.045718193054199, trial: 93
Experiment: 3 -- /home/knut/Documents/project/run_results/run6/xp_003
mse: 3.390287160873413, trial: 27
Experiment: 4 -- /home/knut/Documents/project/run_results/run6/xp_004
mse: 3.295583724975586, trial: 28
Experiment: 5 -- /home/knut/Documents/project/run_results/run6/xp_005
mse: 5.838708877563477, trial: 24
Experiment: 6 -- /home/knut/Documents/project/run_results/run6/xp_006
mse: 1.8179038763046265, trial: 89
Experiment: 7 -- /home/knut/Documents/project/run_results/run6/xp_007
mse: 2.953498125076294, trial: 15
Experiment: 8 -- /home/knut/Documents/project/run_results/run6/xp_008
mse: 3.4157166481018066, trial: 39
Experiment: 9 -- /home/knut/Documents/project/run_results/ru

# Convert into pandas dataframe

In [48]:
df = pd.DataFrame(data)

# Find best trial based on test mse
Store the config of the best model as csv file along with test mse and trial number

In [49]:
data = []
column_names = 1
col_index_of_min = df.idxmin(axis=1)
value_of_min = df.min(axis=1)
for i in range(len(col_index_of_min)):
    path = os.path.join(path_to_run_results, "xp_" + str(i).rjust(3, "0"))
    list_experiment_trials = [f.path for f in os.scandir(path) if f.is_dir()]
    list_experiment_trials.sort(key=lambda x: int(x.split("_")[4]))
    path_to_best_test_mse = list_experiment_trials[col_index_of_min[i]]

    list_trial_checkpoints = [f.path for f in os.scandir(path_to_best_test_mse) if f.is_dir()]
    list_trial_checkpoints.sort(key=lambda x: int(x.split("_")[-1]))
    trial_checkpoint_path = os.path.join(list_trial_checkpoints[-1], "checkpoint")
    trial_analysis = Analysis(path_to_best_test_mse, default_metric=metric, default_mode=mode)
    trial_config = trial_analysis.get_best_config(metric=metric, mode=mode)
    trial_model = config_to_model(config=trial_config, checkpoint_path=trial_checkpoint_path)
    trial_mse = test_model(model=trial_model, batch_size=trial_config["batch_size"])
    assert (trial_mse == value_of_min[i])

    trial_config["mse"] = trial_mse
    trial_config["trial number"] = col_index_of_min[i]

    data.append(trial_config)

config_df = pd.DataFrame(data)

    batch_size  dropout  hidden_layer_width  hidden_layers          l2  \
0            4        0               100.0            7.0  0.00100000   
1            8        0               120.0            8.0  0.00100000   
2            8        0                80.0            5.0  0.00100000   
3            4        0               100.0            5.0  0.04591141   
4           64        0                40.0           10.0  0.04945592   
5           64        0                60.0           10.0  0.04103043   
6            4        0                70.0           10.0  0.07726434   
7           64        0               120.0            2.0  0.00100000   
8           32        0                70.0            7.0  0.09744227   
9            4        0                70.0            6.0  0.01815962   
10           4        0                70.0            3.0  0.00597970   
11           4        0                90.0            4.0  0.00100000   
12           4        0               

# Add mean and variance to test results

In [50]:
df["best"] = df.min(axis=1)
df.loc["mean"] = df.mean(axis=0)
df.loc["var"] = df.var(axis=0)

# Store dataframes as csv

In [51]:
df.to_csv(path_to_csv)
config_df.to_csv(path_to_config_csv)